This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'REMOVED'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import collections

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

def request_FSE_data(API_KEY, start, end):

    # url for FSE, AFX_X
    url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'

    # parameters for request
    parameters = dict(key=API_KEY, start_date=start, end_date=end)

    # GET Request
    res = requests.get(url, params=parameters)

    # return json as dictionary (going one level in since at top level there is only one key/value pair)
    return res.json()['dataset']


AFX_X_test = request_FSE_data(API_KEY, start='2017-01-01', end='2017-01-02')

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
AFX_X_test.keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [6]:
AFX_X_test['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
AFX_X_test['data'][0][0]

'2017-01-02'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# TASK SOLUTIONS

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
2. Convert the returned JSON object into a Python dictionary.

In [8]:
AFX_X_2017 = request_FSE_data(API_KEY, start='2017-01-01', end='2017-12-31')
type(AFX_X_2017)

dict

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
# define function for converting data for a given column into a list (used throughout)
def get_col_data(col_name):
    column_names = AFX_X_2017['column_names']
    index = column_names.index(col_name)
    col_data = []
    
    for record in AFX_X_2017['data']:
        col_data.append(record[index])
        
    return col_data

In [10]:
#define function for removing Nones from a list (used throughout)

def remove_nones(data_list):
    return [n for n in data_list if n]

In [11]:
#get opening prices
opening_prices = get_col_data('Open')

#remove Nones
opening_prices = remove_nones(opening_prices)

#calculate min and max
max_opening = max(opening_prices)
min_opening = min(opening_prices)

print(f'Maximum opening price: ${max_opening}')
print(f'Minimum opening price: ${min_opening}')

Maximum opening price: $53.11
Minimum opening price: $34.0


4. What was the largest change in any one day (based on High and Low price)?

In [12]:
#get high and low prices
high_prices = get_col_data('High')
low_prices = get_col_data('Low')

#initialize empty list to store price deltas
price_deltas = []

#loop through prices and calculate delta for each day
for index in range(len(high_prices)):
    #error handling to skip Nones
    try:
        delta = abs(high_prices[index] - low_prices[index])
    except:
        delta = 0
        
    price_deltas.append(delta)

#get max delta
max_day_change = round(max(price_deltas),2)

print(f'The largest change in any one day was: ${max_day_change}')

The largest change in any one day was: $2.81


5. What was the largest change between any two days (based on Closing Price)?

In [13]:
#get closing prices and remove Nones (assuming that "between any two days" also includes across weekends & holidays)
closing_prices = get_col_data('Close')
closing_prices = remove_nones(closing_prices)

#initialize empty list to store closing price deltas
closing_price_deltas = []

for day in range(1,len(closing_prices)):
    delta = abs(closing_prices[day] - closing_prices[day-1])
    closing_price_deltas.append(delta)

max_2day_change = round(max(closing_price_deltas),2)

print(f'The largest change between any two days was: ${max_2day_change}')

The largest change between any two days was: $2.56


In [14]:
closing_prices = get_col_data('Close')
len(closing_prices)

255

6. What was the average daily trading volume during this year?

In [15]:
#get trading volumes (there are no Nones)
trading_volumes = get_col_data('Traded Volume')

#calculate mean
trading_vol_mean = round(sum(trading_volumes)/len(trading_volumes),2)

print(f'The average (mean) daily trading volume for 2017 was: ${trading_vol_mean}')

The average (mean) daily trading volume for 2017 was: $89124.34


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
def get_median(list):
    sorted_list = sorted(list)
    list_len = len(list)
    
    if list_len % 2 == 0:
        index_low = int(list_len/2 - 0.5)
        index_high = int(list_len/2 + 0.5)
        median = (sorted_list[index_low]+sorted_list[index_high])/2
    else:
        median_index = int(list_len/2 - 0.5)
        median = sorted_list[median_index]
        
    return median

In [17]:
trading_vol_median = get_median(trading_volumes)
print(f'The median daily trading volume for 2017 was: ${trading_vol_median}')

The median daily trading volume for 2017 was: $76286.0
